In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


##have pandas display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
 # Show all columns in df.info()

warnings.filterwarnings('ignore')

%matplotlib inline


In [2]:
df = pd.read_csv("../data/drug_data.csv")


In [3]:
df.head()


,prescription_date,prescription_year,prescription_month,day_of_week,quarter,patient_id,patient_taj,patient_name,birth_date,age_at_prescription,patient_gender,birth_place,medication_name,active_ingredient,atc_code,atc_level1,atc_level2,atc_level3,pharmaceutical_form,active_amount,active_unit,package_quantity,package_unit,ttt_code,ksz_code,package_size,dosage,prescription_status,has_status_data,is_filled,encounter_id,encounter_date,clinic_name,mep_code,medical_specialty,visit_reason,is_admin_visit,doctor_stamp,doctor_name,icd10_codes,diagnosis_names,num_diagnoses,has_diabetes_type1,has_diabetes_type2,has_diabetes_any,has_hyperlipidemia,has_obesity,has_hypothyroidism,has_hyperthyroidism,has_hypertension,has_angina,has_myocardial_infarction,has_ischemic_heart_disease,has_atrial_fibrillation,has_heart_failure,has_cerebrovascular_disease,has_stroke,has_peripheral_vascular_disease,has_asthma,has_copd,has_pneumonia,has_dementia,has_schizophrenia,has_mood_disorder,has_depression,has_anxiety_disorder,has_substance_abuse,has_rheumatoid_arthritis,has_osteoarthritis,has_osteoporosis,has_chronic_kidney_disease,has_acute_kidney_failure,has_cancer,has_breast_cancer,has_lung_cancer,has_colorectal_cancer,has_peptic_ulcer,has_inflammatory_bowel_disease,has_liver_disease,is_pregnant,has_injury,has_infectious_disease,has_blood_disorder,has_neurological_disorder,has_eye_disorder,has_ear_disorder,has_skin_disorder,has_symptoms_signs_abnormal_findings,data_source,prescription_number
0,2023-02-22 17:47:44.000,2023,2,3,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,ALGOPYRIN 500 MG TABLETTA,metamizol-nátrium,N02BB02,N,N02,N02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10x buborékcsomagolásban,1x1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82798,GYŐRI-DANI VERONIKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1
1,2023-02-22 17:36:49.000,2023,2,3,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,LORDESTIN 5 MG FILMTABLETTA,desloratadin,R06AX27,R,R06,R06A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x buborékcsomagolásban,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1
2,2023-02-22 17:36:49.000,2023,2,3,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,VITAMIN D3 PHARMA PATENT 30000 NE FILMTABLETTA,Colecalciferol,A11CC05,A,A11,A11C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2x PVC/PVdC // Alu buborékcsomagolásban, dobozban",utasítás szerint,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1
3,2023-07-06 18:04:29.000,2023,7,4,3,H32001000004,105 568 760,Zámbó József,1966-08-01,56.0,férfi,Nyíregyháza,BÉRES CALCIUM 500 MG FILMTABLETTA,kalcium-karbonát,A12AA04,A,A12,A12A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x hdpe tartályban,D.S.: reggel: 1 tabletta naponta,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45875,SZABÓ LAJOS MIKLÓS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1
4,2023-12-18 13:11:07.000,2023,12,1,4,H32001000004,105 568 760,Zámbó József,1966-08-01,57.0,férfi,Nyíregyháza,BISOPROLOL-RATIOPHARM 5 MG TABLETTA,bisoprolol,C07AB07,C,C07,C07A,tabletta,5.0,mg,30.0,db,210156821.0,30x buborékcsomagolásban,NaN,NaN,NaN,0,NaN,H32001007373,2023-12-18 12:57:39.000,Nyírkáta rendelő,NaN,Belgyógyászat,Akut ellátás,0.0,70282,Dr. Varga Beáta,"I10H0, R0000, Z0130","Magasvérnyomás-betegség (elsődleges), Tachycar...",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45282 entries, 0 to 45281
Data columns (total 90 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   prescription_date                     45282 non-null  object 
 1   prescription_year                     45282 non-null  int64  
 2   prescription_month                    45282 non-null  int64  
 3   day_of_week                           45282 non-null  int64  
 4   quarter                               45282 non-null  int64  
 5   patient_id                            45282 non-null  object 
 6   patient_taj                           45270 non-null  object 
 7   patient_name                          45270 non-null  object 
 8   birth_date                            45270 non-null  object 
 9   age_at_prescription                   45270 non-null  float64
 10  patient_gender                        45270 non-null  object 
 11  birth_place    

In [5]:
## HANDLING DATETIME COLUMNS


In [6]:
df['prescription_date'] = pd.to_datetime(df['prescription_date'])


In [7]:
## drop time component from prescription_date
df['prescription_date_dateonly'] = pd.to_datetime(df['prescription_date'].dt.date)


In [8]:
## month_start = first day of that month (e.g., 2025-02-01)
df['month_start'] = df['prescription_date_dateonly'].dt.to_period('M').dt.start_time


In [9]:
## recalculate prescription_year from prescription_date_dateonly
df['prescription_year'] = df['prescription_date_dateonly'].dt.year
df['prescription_month'] = df['prescription_date_dateonly'].dt.month
df['day_of_week'] = df['prescription_date_dateonly'].dt.dayofweek
df['quarter'] = df['prescription_date_dateonly'].dt.quarter


In [10]:
## Normalize and map prescription_status


In [11]:
## normalize prescription_status, lowercase and trim and all strings

df['prescription_status_norm'] = df['prescription_status'].astype('str').str.lower().str.strip()


In [12]:
map_prescription_status = {
    "3": "filled",
    "4": "filled",
    "5":"filled",
    "8":"filled",
    "dispensed":"filled",
    "6":"not_filled",
    "withdrawn":"not_filled",
     "7":"not_filled",
    "open":"unknown",
    "1":"unknown",
    "2":"unknown",
}

df['prescription_filled_status'] = df['prescription_status_norm'].map(map_prescription_status)


In [13]:
df['prescription_filled_status'].value_counts()


prescription_filled_status
filled        23976
unknown        7579
not_filled     2575
Name: count, dtype: int64

In [14]:
df['prescription_filled_status'].isnull().sum()


np.int64(11152)

In [15]:
##kAdd known_status_flag = 1 if filled/not_filled else 0.
df['known_status_flag'] = df['prescription_filled_status'].isin(['filled','not_filled']).astype(int)


In [16]:
df['known_status_flag'].value_counts()


known_status_flag
1    26551
0    18731
Name: count, dtype: int64

In [17]:
## transform atc_level3 to string and uppercase
df['atc3_code'] = df['atc_level3'].astype('str').str.upper().str.strip()


In [18]:
len(df[df['atc3_code']=='NAN'])


3870

In [19]:
df['medication_name'] = df['medication_name'].str.strip().str.lower()


In [20]:
# 1️⃣ Count how many rows per (date, medication)
counts = (
    df.groupby(['patient_id', 'prescription_date_dateonly', 'medication_name'])
    .size()
    .reset_index(name='count')
)

# 2️⃣ Filter combinations that occur more than once
duplicates = counts[counts['count'] > 1]


In [21]:
df['drug_code_name'] = df['atc3_code'].replace(['', 'NAN', 'NaN', 'nan'], pd.NA)

# Group by patient and compute flags
consistency = (
    df.groupby('patient_id')
      .agg(
          has_atc3=('drug_code_name', lambda x: x.notna().any()),
          missing_atc3=('drug_code_name', lambda x: x.isna().any())
      )
      .reset_index()
)

# Patients with both present AND missing ATC3
mixed = consistency[consistency['has_atc3'] & consistency['missing_atc3']]

mixed.head()


,patient_id,has_atc3,missing_atc3
8,H32001000021,True,True
13,H32001000037,True,True
17,H32001000048,True,True
25,H32001000064,True,True
26,H32001000065,True,True


In [22]:
## create rx_line_id
df['drug_code_name'] = df['drug_code_name'].fillna(df['medication_name'])

df['rx_line_id'] = df['patient_id'].astype(str)+'_'+df['drug_code_name']+'_'+df['prescription_date_dateonly'].astype(str)


In [23]:
df = df.sort_values(['rx_line_id','prescription_date'])


In [24]:
df.head()


,prescription_date,prescription_year,prescription_month,day_of_week,quarter,patient_id,patient_taj,patient_name,birth_date,age_at_prescription,patient_gender,birth_place,medication_name,active_ingredient,atc_code,atc_level1,atc_level2,atc_level3,pharmaceutical_form,active_amount,active_unit,package_quantity,package_unit,ttt_code,ksz_code,package_size,dosage,prescription_status,has_status_data,is_filled,encounter_id,encounter_date,clinic_name,mep_code,medical_specialty,visit_reason,is_admin_visit,doctor_stamp,doctor_name,icd10_codes,diagnosis_names,num_diagnoses,has_diabetes_type1,has_diabetes_type2,has_diabetes_any,has_hyperlipidemia,has_obesity,has_hypothyroidism,has_hyperthyroidism,has_hypertension,has_angina,has_myocardial_infarction,has_ischemic_heart_disease,has_atrial_fibrillation,has_heart_failure,has_cerebrovascular_disease,has_stroke,has_peripheral_vascular_disease,has_asthma,has_copd,has_pneumonia,has_dementia,has_schizophrenia,has_mood_disorder,has_depression,has_anxiety_disorder,has_substance_abuse,has_rheumatoid_arthritis,has_osteoarthritis,has_osteoporosis,has_chronic_kidney_disease,has_acute_kidney_failure,has_cancer,has_breast_cancer,has_lung_cancer,has_colorectal_cancer,has_peptic_ulcer,has_inflammatory_bowel_disease,has_liver_disease,is_pregnant,has_injury,has_infectious_disease,has_blood_disorder,has_neurological_disorder,has_eye_disorder,has_ear_disorder,has_skin_disorder,has_symptoms_signs_abnormal_findings,data_source,prescription_number,prescription_date_dateonly,month_start,prescription_status_norm,prescription_filled_status,known_status_flag,atc3_code,drug_code_name,rx_line_id
2,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,vitamin d3 pharma patent 30000 ne filmtabletta,Colecalciferol,A11CC05,A,A11,A11C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2x PVC/PVdC // Alu buborékcsomagolásban, dobozban",utasítás szerint,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,A11C,A11C,H32001000001_A11C_2023-02-22
0,2023-02-22 17:47:44,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,algopyrin 500 mg tabletta,metamizol-nátrium,N02BB02,N,N02,N02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10x buborékcsomagolásban,1x1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82798,GYŐRI-DANI VERONIKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,6,not_filled,1,N02B,N02B,H32001000001_N02B_2023-02-22
1,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,lordestin 5 mg filmtabletta,desloratadin,R06AX27,R,R06,R06A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x buborékcsomagolásban,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,R06A,R06A,H32001000001_R06A_2023-02-22
8,2023-12-18 13:09:22,2023,12,0,4,H32001000004,105 568 760,Zámbó József,1966-08-01,57.0,férfi,Nyíregyháza,esomeprazol sandoz 40 mg gyomornedv-ellenálló ...,esomeprazol,A02BC05,A,A02,A02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28x buborékcsomagolásban alu//alu,D.S.: reggel: 1 tabletta naponta,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45875,SZABÓ LAJOS MIKLÓS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,2,2023-12-18,2023-12-01,1,unknown,0,A02B,A02B,H32001000004_A02B_2023-12-18
9,2023-12-18 13:09:22,2023,12,0,4,H32001000004,105 568 760,Zámbó József,1966-08-01,57.0,férfi,Nyíregyháza,es

In [25]:
df['status_norm'] = (
    df['prescription_filled_status']
      .astype(str)
      .str.strip()
      .str.lower()
)
FINAL_STATUSES = ['filled', 'not_filled']


In [26]:
def get_day_summary(group):
    n_filled     = (group['status_norm'] == 'filled').sum()
    n_not_filled = (group['status_norm'] == 'not_filled').sum()

    if n_not_filled > 0:
        final_status = 'not_filled'
    elif n_filled > 0:
        final_status = 'filled'
    else:
        final_status = 'unknown'
    
    # pick latest representative row
    if final_status == 'unknown':
        rep = group.iloc[group['prescription_date'].argmax()]
    else:
        rep = group[group['status_norm'] == final_status] \
                  .iloc[group[group['status_norm'] == final_status]['prescription_date'].argmax()]

    return pd.Series({
        'final_status_day': final_status,
        'day_entries_merged_count': len(group),
        'packages_that_day': n_filled,
        'had_conflict_final_same_day': int((n_filled > 0) and (n_not_filled > 0)),
        'prescription_date_rep': rep['prescription_date'],
        'known_status_flag_rep': rep['known_status_flag']
    })

# Attach day-level fields directly to df
df_day_summary = (
    df.groupby(['rx_line_id', 'prescription_date_dateonly'])
      .apply(get_day_summary)
      .reset_index()
)


In [27]:
df = df.merge(
    df_day_summary,
    on=['rx_line_id', 'prescription_date_dateonly'],
    how='left'
)

df.head()


,prescription_date,prescription_year,prescription_month,day_of_week,quarter,patient_id,patient_taj,patient_name,birth_date,age_at_prescription,patient_gender,birth_place,medication_name,active_ingredient,atc_code,atc_level1,atc_level2,atc_level3,pharmaceutical_form,active_amount,active_unit,package_quantity,package_unit,ttt_code,ksz_code,package_size,dosage,prescription_status,has_status_data,is_filled,encounter_id,encounter_date,clinic_name,mep_code,medical_specialty,visit_reason,is_admin_visit,doctor_stamp,doctor_name,icd10_codes,diagnosis_names,num_diagnoses,has_diabetes_type1,has_diabetes_type2,has_diabetes_any,has_hyperlipidemia,has_obesity,has_hypothyroidism,has_hyperthyroidism,has_hypertension,has_angina,has_myocardial_infarction,has_ischemic_heart_disease,has_atrial_fibrillation,has_heart_failure,has_cerebrovascular_disease,has_stroke,has_peripheral_vascular_disease,has_asthma,has_copd,has_pneumonia,has_dementia,has_schizophrenia,has_mood_disorder,has_depression,has_anxiety_disorder,has_substance_abuse,has_rheumatoid_arthritis,has_osteoarthritis,has_osteoporosis,has_chronic_kidney_disease,has_acute_kidney_failure,has_cancer,has_breast_cancer,has_lung_cancer,has_colorectal_cancer,has_peptic_ulcer,has_inflammatory_bowel_disease,has_liver_disease,is_pregnant,has_injury,has_infectious_disease,has_blood_disorder,has_neurological_disorder,has_eye_disorder,has_ear_disorder,has_skin_disorder,has_symptoms_signs_abnormal_findings,data_source,prescription_number,prescription_date_dateonly,month_start,prescription_status_norm,prescription_filled_status,known_status_flag,atc3_code,drug_code_name,rx_line_id,status_norm,final_status_day,day_entries_merged_count,packages_that_day,had_conflict_final_same_day,prescription_date_rep,known_status_flag_rep
0,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,vitamin d3 pharma patent 30000 ne filmtabletta,Colecalciferol,A11CC05,A,A11,A11C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2x PVC/PVdC // Alu buborékcsomagolásban, dobozban",utasítás szerint,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,A11C,A11C,H32001000001_A11C_2023-02-22,filled,filled,1,1,0,2023-02-22 17:36:49,1
1,2023-02-22 17:47:44,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,algopyrin 500 mg tabletta,metamizol-nátrium,N02BB02,N,N02,N02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10x buborékcsomagolásban,1x1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82798,GYŐRI-DANI VERONIKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,6,not_filled,1,N02B,N02B,H32001000001_N02B_2023-02-22,not_filled,not_filled,1,0,0,2023-02-22 17:47:44,1
2,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,lordestin 5 mg filmtabletta,desloratadin,R06AX27,R,R06,R06A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x buborékcsomagolásban,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,R06A,R06A,H32001000001_R06A_2023-02-22,filled,filled,1,1,0,2023-02-22 17:36:49,1
3,2023-12-18 13:09:22,2023,12,0,4,H32001000004,105 568 760,Zámbó József,1966-08-01,57.0,férfi,Nyíregyháza,esomeprazol sandoz 40 mg gyomornedv-ellenálló ...,esomeprazol,A02BC05,A,A02,A02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28x buborékcsomagolásban alu//alu,D.S.: reggel: 1 tabletta naponta,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45875,SZABÓ LAJOS MIKLÓS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [28]:
def get_line_summary(group):
    finals = group[group['final_status_day'].isin(['filled', 'not_filled'])]

    if len(finals) == 0:
        chosen = group.loc[group['prescription_date_rep'].idxmax()]
    else:
        chosen = finals.loc[finals['prescription_date_rep'].idxmax()]
    
    return pd.Series({
        'final_status': chosen['final_status_day'],
        'prescription_date_final': chosen['prescription_date_dateonly'],
        'prescription_datetime_final': chosen['prescription_date_rep'],
        'entries_merged_total': group['day_entries_merged_count'].sum(),
        'had_conflict_any_day': int(group['had_conflict_final_same_day'].any()),
        'packages_total': group['packages_that_day'].sum(),
        'known_status_flag_final': chosen['known_status_flag_rep'],
    })

df_line_summary = (
    df.groupby('rx_line_id')
      .apply(get_line_summary)
      .reset_index()
)


In [29]:
df = df.merge(df_line_summary, on='rx_line_id', how='left')


In [30]:
df = df.drop_duplicates(subset=['rx_line_id'])


In [31]:
df.shape


(27206, 112)

In [32]:
df.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
Index: 27206 entries, 0 to 45281
Data columns (total 112 columns):
 #    Column                                Dtype         
---   ------                                -----         
 0    prescription_date                     datetime64[ns]
 1    prescription_year                     int32         
 2    prescription_month                    int32         
 3    day_of_week                           int32         
 4    quarter                               int32         
 5    patient_id                            object        
 6    patient_taj                           object        
 7    patient_name                          object        
 8    birth_date                            object        
 9    age_at_prescription                   float64       
 10   patient_gender                        object        
 11   birth_place                           object        
 12   medication_name                       object        
 13   acti

In [33]:
df['final_status'] = (
    df['final_status']
      .astype(str)
      .str.strip()
      .str.lower()
)


In [34]:
## biweek_start = 1st day if day 1-14, 15th day if day 15+
# Period 1: 1st-14th → biweek_start = 1st of month
# Period 2: 15th-end → biweek_start = 15th of month

df['biweek_start'] = df['prescription_date_dateonly'].apply(
    lambda x: x.replace(day=1) if x.day <= 14 else x.replace(day=15)
)


In [35]:
df[['prescription_date_dateonly', 'biweek_start']].head(20)


,prescription_date_dateonly,biweek_start
0,2023-02-22,2023-02-15
1,2023-02-22,2023-02-15
2,2023-02-22,2023-02-15
3,2023-12-18,2023-12-15
6,2023-07-06,2023-07-01
7,2023-07-06,2023-07-01
8,2023-07-06,2023-07-01
9,2023-12-18,2023-12-15
10,2023-07-06,2023-07-01
11,2023-12-18,2023-12-15


In [36]:
known_vals = ['filled', 'not_filled']

biweekly = (
    df
    .groupby('biweek_start')
    .agg(
        total_rx=('rx_line_id', 'size'),
        # how many lines have known final status
        known_status_rx=('known_status_flag_final', 'sum'),
        # how many are filled
        filled_rx=('final_status', lambda x: (x == 'filled').sum())
    )
    .reset_index()
)


In [37]:
biweekly['status_coverage'] = biweekly['known_status_rx'] / biweekly['total_rx']
biweekly.loc[biweekly['total_rx'] == 0, 'status_coverage'] = pd.NA

# refill_adherence = filled_rx / known_status_rx  (NA if known_status_rx == 0)
biweekly['refill_adherence'] = biweekly['filled_rx'] / biweekly['known_status_rx']
biweekly.loc[biweekly['known_status_rx'] == 0, 'refill_adherence'] = pd.NA


In [38]:
biweekly.head(10)


,biweek_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence
0,2023-02-15,6,3,2,0.500000,0.666667
1,2023-03-01,99,95,87,0.959596,0.915789
2,2023-03-15,195,171,138,0.876923,0.807018
3,2023-04-01,205,194,177,0.946341,0.912371
4,2023-04-15,294,256,235,0.870748,0.917969
5,2023-05-01,504,380,341,0.753968,0.897368
6,2023-05-15,488,343,323,0.702869,0.941691
7,2023-06-01,400,269,233,0.672500,0.866171
8,2023-06-15,397,272,215,0.685139,0.790441
9,2023-07-01,496,368,337,0.741935,0.915761


In [39]:
biweekly['unique_patients'] = biweekly['biweek_start'].map(
    df.groupby('biweek_start')['patient_id'].nunique()
)


In [40]:
biweekly.head(10)


,biweek_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients
0,2023-02-15,6,3,2,0.500000,0.666667,3
1,2023-03-01,99,95,87,0.959596,0.915789,17
2,2023-03-15,195,171,138,0.876923,0.807018,41
3,2023-04-01,205,194,177,0.946341,0.912371,28
4,2023-04-15,294,256,235,0.870748,0.917969,54
5,2023-05-01,504,380,341,0.753968,0.897368,98
6,2023-05-15,488,343,323,0.702869,0.941691,100
7,2023-06-01,400,269,233,0.672500,0.866171,96
8,2023-06-15,397,272,215,0.685139,0.790441,97
9,2023-07-01,496,368,337,0.741935,0.915761,104


In [41]:
df['patient_gender'].value_counts()


patient_gender
nő       19386
férfi     7808
Name: count, dtype: int64

In [42]:
## map gender to 'female' 'male' and 'unknown'
gender_map = {
    'nő': 'female',
    'férfi': 'male',
    'nan': 'unknown',
}

df['gender_norm'] = df['patient_gender'].str.strip().str.lower().map(gender_map)


In [43]:
df['gender_norm'].value_counts(dropna=False)


gender_norm
female    19386
male       7808
NaN          12
Name: count, dtype: int64

In [44]:
gender_stats = (
    df
    .groupby('biweek_start')
    .agg(
        female_count=('gender_norm', lambda x: (x == 'female').sum()),
        known_gender_count=('gender_norm', lambda x: x.notna().sum())
    )
)

gender_stats['pct_female'] = (
    gender_stats['female_count'] / gender_stats['known_gender_count']
)

# If no known gender that biweek → NA
gender_stats.loc[gender_stats['known_gender_count'] == 0, 'pct_female'] = pd.NA


In [45]:
biweekly['pct_female'] = biweekly['biweek_start'].map(gender_stats['pct_female'])


In [46]:
biweekly.head(10)


,biweek_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female
0,2023-02-15,6,3,2,0.500000,0.666667,3,1.000000
1,2023-03-01,99,95,87,0.959596,0.915789,17,0.848485
2,2023-03-15,195,171,138,0.876923,0.807018,41,0.558974
3,2023-04-01,205,194,177,0.946341,0.912371,28,0.858537
4,2023-04-15,294,256,235,0.870748,0.917969,54,0.724490
5,2023-05-01,504,380,341,0.753968,0.897368,98,0.636905
6,2023-05-15,488,343,323,0.702869,0.941691,100,0.733607
7,2023-06-01,400,269,233,0.672500,0.866171,96,0.817500
8,2023-06-15,397,272,215,0.685139,0.790441,97,0.670025
9,2023-07-01,496,368,337,0.741935,0.915761,104,0.760081


In [47]:
age_stats = (
    df.groupby('biweek_start')
    .agg(
        median_age=('age_at_prescription', lambda x: x.dropna().median())
    )
)


In [48]:
biweekly['median_age'] = biweekly['biweek_start'].map(age_stats['median_age'])


In [49]:
biweekly.tail(10)


,biweek_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female,median_age
56,2025-06-15,242,0,0,0.0,NaN,125,0.690083,58.0
57,2025-07-01,129,0,0,0.0,NaN,82,0.643411,60.0
58,2025-07-15,198,0,0,0.0,NaN,111,0.636364,53.0
59,2025-08-01,164,0,0,0.0,NaN,97,0.689024,58.0
60,2025-08-15,179,0,0,0.0,NaN,108,0.664804,58.0
61,2025-09-01,153,0,0,0.0,NaN,92,0.751634,60.0
62,2025-09-15,219,0,0,0.0,NaN,135,0.684932,56.0
63,2025-10-01,216,0,0,0.0,NaN,118,0.689815,56.0
64,2025-10-15,252,0,0,0.0,NaN,154,0.773810,54.0
65,2025-11-01,135,0,0,0.0,NaN,66,0.674074,56.0


In [50]:
biweekly['low_coverage_flag'] = (biweekly['status_coverage'] < 0.70).astype(int)


In [51]:
biweekly.head(10)


,biweek_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female,median_age,low_coverage_flag
0,2023-02-15,6,3,2,0.500000,0.666667,3,1.000000,26.0,1
1,2023-03-01,99,95,87,0.959596,0.915789,17,0.848485,58.0,0
2,2023-03-15,195,171,138,0.876923,0.807018,41,0.558974,56.0,0
3,2023-04-01,205,194,177,0.946341,0.912371,28,0.858537,54.0,0
4,2023-04-15,294,256,235,0.870748,0.917969,54,0.724490,59.0,0
5,2023-05-01,504,380,341,0.753968,0.897368,98,0.636905,55.0,0
6,2023-05-15,488,343,323,0.702869,0.941691,100,0.733607,61.0,0
7,2023-06-01,400,269,233,0.672500,0.866171,96,0.817500,60.0,1
8,2023-06-15,397,272,215,0.685139,0.790441,97,0.670025,62.0,1
9,2023-07-01,496,368,337,0.741935,0.915761,104,0.760081,61.0,0


In [52]:
biweekly_overall = biweekly.copy()


In [53]:
biweekly_overall['biweek_start'] = pd.to_datetime(biweekly_overall['biweek_start'])


In [54]:
# Create full bi-weekly calendar (1st and 15th of each month)
min_date = biweekly_overall['biweek_start'].min()
max_date = biweekly_overall['biweek_start'].max()

# Get all months in range
monthly_range = pd.date_range(start=min_date, end=max_date, freq='MS')

# For each month, create both bi-weekly periods (1st and 15th)
biweekly_calendar = []
for month_start in monthly_range:
    biweekly_calendar.append(month_start)  # 1st of month
    biweekly_calendar.append(month_start.replace(day=15))  # 15th of month

# Also add 15th of the last month if max_date is on the 15th
if max_date.day >= 15:
    last_15th = max_date.replace(day=15)
    if last_15th not in biweekly_calendar:
        biweekly_calendar.append(last_15th)

biweekly_calendar = pd.DatetimeIndex(sorted(set(biweekly_calendar)))


In [55]:
biweekly_calendar


DatetimeIndex(['2023-03-01', '2023-03-15', '2023-04-01', '2023-04-15',
               '2023-05-01', '2023-05-15', '2023-06-01', '2023-06-15',
               '2023-07-01', '2023-07-15', '2023-08-01', '2023-08-15',
               '2023-09-01', '2023-09-15', '2023-10-01', '2023-10-15',
               '2023-11-01', '2023-11-15', '2023-12-01', '2023-12-15',
               '2024-01-01', '2024-01-15', '2024-02-01', '2024-02-15',
               '2024-03-01', '2024-03-15', '2024-04-01', '2024-04-15',
               '2024-05-01', '2024-05-15', '2024-06-01', '2024-06-15',
               '2024-07-01', '2024-07-15', '2024-08-01', '2024-08-15',
               '2024-09-01', '2024-09-15', '2024-10-01', '2024-10-15',
               '2024-11-01', '2024-11-15', '2024-12-01', '2024-12-15',
               '2025-01-01', '2025-01-15', '2025-02-01', '2025-02-15',
               '2025-03-01', '2025-03-15', '2025-04-01', '2025-04-15',
               '2025-05-01', '2025-05-15', '2025-06-01', '2025-06-15',
      

In [56]:
biweekly_full = (
    biweekly_overall
    .set_index('biweek_start')
    .reindex(biweekly_calendar)   # reindex to full biweekly calendar
)


In [57]:
biweekly_full.index.name = 'biweek_start'


In [58]:
count_cols = ['total_rx', 'known_status_rx', 'filled_rx', 'unique_patients']

for col in count_cols:
    if col in biweekly_full.columns:
        biweekly_full[col] = biweekly_full[col].fillna(0).astype(int)


In [59]:
if {'status_coverage', 'refill_adherence'}.issubset(biweekly_full.columns):
    # status_coverage = known_status_rx / total_rx
    biweekly_full['status_coverage'] = (
        biweekly_full['known_status_rx'] / biweekly_full['total_rx']
    )
    biweekly_full.loc[biweekly_full['total_rx'] == 0, 'status_coverage'] = pd.NA

    # refill_adherence = filled_rx / known_status_rx
    biweekly_full['refill_adherence'] = (
        biweekly_full['filled_rx'] / biweekly_full['known_status_rx']
    )
    biweekly_full.loc[biweekly_full['known_status_rx'] == 0, 'refill_adherence'] = pd.NA


In [60]:
biweekly_full = biweekly_full.reset_index()


In [61]:
biweekly_full.shape


(66, 10)

In [62]:
## drop rows from 2025-02-01 and onwards
biweekly_full = biweekly_full[biweekly_full['biweek_start'] < '2025-02-01']
biweekly_full.head(10)


,biweek_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female,median_age,low_coverage_flag
0,2023-03-01,99,95,87,0.959596,0.915789,17,0.848485,58.0,0.0
1,2023-03-15,195,171,138,0.876923,0.807018,41,0.558974,56.0,0.0
2,2023-04-01,205,194,177,0.946341,0.912371,28,0.858537,54.0,0.0
3,2023-04-15,294,256,235,0.870748,0.917969,54,0.724490,59.0,0.0
4,2023-05-01,504,380,341,0.753968,0.897368,98,0.636905,55.0,0.0
5,2023-05-15,488,343,323,0.702869,0.941691,100,0.733607,61.0,0.0
6,2023-06-01,400,269,233,0.672500,0.866171,96,0.817500,60.0,1.0
7,2023-06-15,397,272,215,0.685139,0.790441,97,0.670025,62.0,1.0
8,2023-07-01,496,368,337,0.741935,0.915761,104,0.760081,61.0,0.0
9,2023-07-15,268,187,166,0.697761,0.887701,69,0.757463,54.0,1.0


In [64]:
biweekly_full.shape


(46, 10)

In [65]:
biweekly_full.to_csv('../data/biweekly_overall.csv', index=False)